In [89]:
# imports
import os
import json
import pprint
import pandas as pd

# constants
CURRENT_DIR = os.getcwd()
DATA_DIR = os.path.abspath(os.path.join(CURRENT_DIR, 'data'))
MOJO_DIR = os.path.join(DATA_DIR, 'boxofficemojo')
metacritic_DIR = os.path.join(DATA_DIR, 'metacritic')

##LOADING DATA

In [90]:
def load_mojo_data():
    file_contents = os.listdir(MOJO_DIR)

    movie_dicts_mojo = []

    for filename in file_contents:
        filepath = os.path.join(MOJO_DIR, filename)

        with open(filepath, 'r') as movie_file:
            movie_data = json.load(movie_file)

        movie_dicts_mojo.append(movie_data)

    print "Parsed %i movies from %i files" % (len(movie_dicts_mojo),
                                              len(file_contents))
    return movie_dicts_mojo

def load_metacritic_data():
    file_contents = os.listdir(metacritic_DIR)

    movie_dicts_meta = []

    for filename in file_contents:
        filepath = os.path.join(metacritic_DIR, filename)
        
        with open(filepath, 'r') as movie_file:
            movie_data = json.load(movie_file)
        if type(movie_data)==dict:
            movie_dicts_meta.append(movie_data)

    print "Parsed %i movies from %i files" % (len(movie_dicts_meta),
                                              len(file_contents))
    return movie_dicts_meta

In [91]:
#import loaddata
#import pandas as pd
movie_dicts_mojo = load_mojo_data()
movie_df_mojo = pd.DataFrame(movie_dicts_mojo)
movie_dicts_meta = load_metacritic_data()
movie_df_meta = pd.DataFrame(movie_dicts_meta)

Parsed 3728 movies from 3728 files
Parsed 4765 movies from 4767 files


In [92]:
#clean title

#metacric_df2['title']=metacric_df2.title_Upper.str.strip() 
#movie_df_meta.drop('unable to retrieve', axis=1, inplace=True) 
#metacric_df1=metacric_df.drop(metacric_df.index[[4764]]) 
movie_df_meta['title_Upper']=movie_df_meta.title.str.upper() 
movie_df_meta['title_new']=movie_df_meta.title_Upper.str.replace(r'\W+','',case=False)
movie_df_mojo['title_Upper']=movie_df_mojo.title.str.upper() 
movie_df_mojo['title_new']=movie_df_mojo.title_Upper.str.replace(r'\W+','',case=False)

In [93]:
#clean director
movie_df_meta['director_upper']=movie_df_meta.director.str.upper() 
movie_df_meta['director_new']=movie_df_meta.director_upper.str.replace(r'\W+','',case=False)
movie_df_mojo['director_upper']=movie_df_mojo.director.str.upper() 
movie_df_mojo['director_new']=movie_df_mojo.director_upper.str.replace(r'\W+','',case=False)

In [94]:

#Clean year
movie_df_mojo['year']=pd.to_numeric(movie_df_mojo['year'])
movie_df_meta['year']=pd.to_numeric(movie_df_meta['year'])

In [95]:

# Join based on title and Year
movie_merge= pd.merge(movie_df_mojo,movie_df_meta, on=['title_new', 'year'], how="inner")

In [96]:
#check for duplicates
#movie_merge[movie_merge.duplicated(['title_new'])][['director_x','director_new_x','director_y']]
# clean_movie_df_dir = movie_df_dir[related_columns].dropna()

In [97]:
data_description=movie_merge.describe()
data_description.to_csv('./data_description.csv')

In [98]:
movie_merge.columns

Index([           u'alt_title',           u'director_x',
             u'domestic_gross',            u'mojo_slug',
        u'opening_per_theater', u'opening_weekend_take',
          u'production_budget', u'release_date_limited',
          u'release_date_wide',              u'title_x',
             u'widest_release',      u'worldwide_gross',
                       u'year',        u'title_Upper_x',
                  u'title_new',     u'director_upper_x',
             u'director_new_x',             u'complete',
                 u'director_y',                u'genre',
            u'metacritic_page',            u'metascore',
         u'num_critic_reviews',     u'num_user_ratings',
           u'num_user_reviews',               u'rating',
               u'release_date',      u'runtime_minutes',
                     u'studio',              u'title_y',
         u'unable to retrieve',           u'user_score',
              u'title_Upper_y',     u'director_upper_y',
             u'director_new_y']

In [99]:

movie_merge1=movie_merge.drop('director_new_x',axis=1)

In [100]:
movie_merge2=movie_merge1.drop('unable to retrieve',axis=1)

In [101]:
len(movie_merge2.release_date_limited.unique())


250

In [103]:
movie_merge3=movie_merge2.drop('title_Upper_x',axis=1)
movie_merge4=movie_merge3.drop('title_Upper_y',axis=1)
movie_merge4=movie_merge3.drop('title_Upper_y',axis=1)
movie_merge6=movie_merge4.drop('director_upper_x',axis=1)

In [186]:
movie_merge6['rating']=movie_merge6.rating.str.replace('Not Rated','Unrated')
movie_merge6['rating']=movie_merge6.rating.str.replace('PG--13','PG-13')
movie_merge6['rating']=movie_merge6.rating.str.replace('TV-14','TV')
movie_merge6['rating']=movie_merge6.rating.str.replace('TV-G','TV')
movie_merge6['rating']=movie_merge6.rating.str.replace('TV-MA','TV')
movie_merge6['rating']=movie_merge6.rating.str.replace('TV-PG','TV')
movie_merge6['rating']=movie_merge6.rating.str.replace('TV-Y7','TV')
movie_merge6['rating']=movie_merge6.rating.str.replace(r'\s+( +\.)|#','Unrated').replace('','Unrated')
movie_merge6['rating']=movie_merge6.rating.str.replace('Open','Unrated')
movie_merge6['rating']=movie_merge6.rating.str.replace('Approved','PG')
movie_merge6['rating']=movie_merge6.rating.str.replace('GP','G')
movie_merge6['rating']=movie_merge6.rating.str.replace('M','PG')

In [187]:
complete_dummies = pd.get_dummies(movie_merge6['complete'])

In [188]:
movie_merge7=pd.concat([movie_merge6,complete_dummies],axis=1)

In [189]:
pd.crosstab(movie_merge6.complete,movie_merge6.rating)

rating,G,NC-17,PG,PG-13,R,TV,Unrated
complete,,,,,,,
False,2,0,14,34,67,1,181
True,62,27,362,820,1360,73,345


In [190]:
rating_dummies = pd.get_dummies(movie_merge7['rating'])

In [191]:
movie_merge8=pd.concat([movie_merge7,rating_dummies],axis=1)

In [194]:
movie_merge8.dtypes

alt_title                object
director_x               object
domestic_gross          float64
mojo_slug                object
opening_per_theater     float64
opening_weekend_take    float64
production_budget       float64
release_date_limited     object
release_date_wide        object
title_x                  object
widest_release          float64
worldwide_gross         float64
year                    float64
title_new                object
complete                 object
director_y               object
genre                    object
metacritic_page          object
metascore               float64
num_critic_reviews       object
num_user_ratings        float64
num_user_reviews         object
rating                   object
release_date             object
runtime_minutes         float64
studio                   object
title_y                  object
user_score               object
director_new_y           object
False                   float64
True                    float64
G       

In [196]:
movie_merge8.release_date_wide

0       2008-03-07
1       2000-11-22
2       1999-03-31
3       2012-09-14
4       2013-02-22
5       2007-08-17
6       2011-01-28
7       2009-03-27
8       2011-04-29
9       2004-04-23
10      2014-04-18
11      1999-08-27
12      2006-07-28
13      2007-06-22
14      2006-03-03
15      2009-04-17
16      1968-04-03
17      2009-11-13
18      2012-08-24
19      2005-08-05
20      2013-06-14
21      2008-03-28
22      2013-03-01
23      2003-11-21
24      2012-03-16
25      2002-12-19
26      2008-01-18
27      2000-04-14
28      2003-06-27
29      2007-05-11
           ...    
3318    2008-12-19
3319    2004-09-24
3320    2010-12-17
3321    2013-01-25
3322    2010-09-24
3323    2000-11-10
3324    2008-06-06
3325    2004-01-30
3326    2007-06-22
3327    2006-07-14
3328    2011-12-16
3329    1988-08-12
3330    2009-12-18
3331    2013-08-23
3332    2011-04-08
3333    2012-06-15
3334    2005-11-23
3335    2010-01-08
3336    2012-08-03
3337    1998-12-18
3338    2013-08-16
3339    2014

In [205]:
quarter_dummies.head()

,1.0,2.0,3.0,4.0
0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0


In [211]:

import statsmodels.api as sm

movie_merge8.reset_index(inplace=True)
movie_merge8['release_date_wide'] = pd.to_datetime(movie_merge8['release_date_wide'])
movie_merge8 = movie_merge8.set_index('release_date_wide')
s=sm.tsa.seasonal_decompose(movie_merge8.domestic_gross.dropna())

ValueError: You must specify a freq or x must be a pandas object with a timeseries index